# Deuxième partie : Détection des tumeurs cérébrales avec YOLOv8

Cette partie consiste à classer les images de tumeurs cérébrales et à localiser les tumeurs à l'aide du modèle YOLOv8.

In [ ]:
# Imports et configuration initiale

## 1. Afficher un échantillon d'images pour chaque classe avec les boîtes englobantes

## 2. Créer un dossier pour stocker les données après filtrage

## 3. Filtrer et copier les images et labels

Pour chaque image :
- Vérifier si un fichier .txt correspondant existe dans le répertoire des labels
- Si un label est trouvé : copier l'image et le label vers le dossier approprié (train/valid/test)
- Si aucun label n'est trouvé : afficher un avertissement et sauter l'image

## 4. Créer le fichier data.yaml sans augmentations

Ce fichier contient :
- Les chemins des dossiers d'entraînement, de validation et de test
- Le nombre de classes et le nom de chaque classe
- Configuration sans augmentations de données

## 5. Créer le fichier data2.yaml avec augmentations

Ce fichier contient la même structure que data.yaml mais avec des augmentations de données activées

## 6. Compter le nombre d'images et d'étiquettes

Compter les images et labels présents dans les ensembles d'entraînement et de validation

## 7. Vérifier la correspondance images-labels et nettoyer les données

- Vérifier que chaque image possède un label correspondant
- Supprimer les images sans label
- Supprimer les labels sans image correspondante

## 8. Entraîner le modèle YOLOv8

Lancer l'entraînement du modèle YOLO en définissant les hyperparamètres appropriés

## 9. Évaluer et tester le modèle

Évaluer et tester le modèle après l'entraînement pour mesurer :
- Les performances
- La précision
- La capacité à généraliser sur des données inédites

## 10. Sauvegarder le modèle entraîné

Sauvegarder le modèle entraîné pour une utilisation future